# Setup_system
> @author: Jiheng Duan

## About

This is a demonstration file to show how to set up system, change system parameter, get data from the defined system. 

Our simulator is based on `QuTip`, which means the user should have some fundamental skills on `QuTip`.

### Required Import

**The following thing must be import.**

In the current folder (despite `~\Tutorial\arb_qubit_tutorial\`), the system structure are based on file `~\System\transmon_system.py`. For using arbitrary qubit system structure, please move to the demonstration file under `~\Tutorial\arb_qubit_tutorial\`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
# This on is a path correcting code, which is used to find the path of qusim.
sys.path.append('../..')

import qusim.System.transmon_system as qs
# Some useful tools
import qusim.Instruments.tools as tools

### Optional import

**The following import are optional**

In [2]:
# Some intrinsic plotting function
import qusim.DataPlot.plot_lib as pl

# Print the full output
np.set_printoptions(threshold=sys.maxsize)

# auto reload
%load_ext autoreload
%autoreload 2 

# variable name -> string
def var_name2str(variable):
    def nested_var_name2str(value):
        if isinstance(value, list):
            return [nested_var_name2str(item) for item in value]
        else:
            for name in globals():
                if eval(name) == value:
                    return name
            return str(value)
    return nested_var_name2str(variable)

## Initializing the system

The first thing we need to know first is what the model of our system. The `transmon_system.py` uses `QuTip.Qobj` to define the corresponding physical quantities such as Hamiltonian, state ket, density matrix, etc.

We use `qs.qubit_system(N, q_dim, w, alpha, r, gamma_list)` to setup a class to setup the system. 

>#### `N: int` or `None`. 
>
> `N` controls the maximum excitation level of the system, which is help to reduce the total simulated space in order to save computing time and resources. Usually, `N` is set to be `None`, which means turn off the restriction of maximum excitation levels.
> #### `q_dim: list`
>
> `q_dim` is a list containing the dimension of each individual quantum system. The element inside the list should be `int`. The `len(q_dim)` should be the number of subsystem (i.e., number of qubit, coupler, and resonator).
>
> #### `w: list`
>
> `w` is a list containing the transition frequency between $0$ and $1$ level $\omega_{0\rightarrow 1}$. The `len(w)` should be the number of subsystem (i.e., number of qubit, coupler, and resonator).
>
> #### `alpha: list`
>
> `alpha` is a list containing the anharmonicity of each sub system.
>
> #### `r = 0`
>
> Default is zero. `r` is a 2D-list containing all the coupling strength information. Usually, we define a dictionary first, such as
```
r_dic = {
    "r12": 0.05,
    "r13": -0.005,
    "r23": 0.05
}
```
> where `"rij"` means the coupling strength between subsystem `i` and `j`, and notice that `i<j` always. Then, use the `tools.r2matrix()` to convert this dictionary into a 2D-array and serves as an input of `qs.qubit_system`.
>
> **Please Notice that you could only define the non zero interaction.**. The missing interaction will be filled with zero automatically.
>
> #### `gamma_list = None`
> Default is zero. It represents the collapse operators of each subsystem. Please notice if you want to define the collapse operator for any on of the subsystem, you should write down a full list. I.e., for two qubit system, once you want to define a `gamma_down` only for qubit 2, you should write the `gamma_list` as
```
gamma_list = [
    {
        "up": 0,
        "down": 0,
        "z": 0
    },
    {
        "up": 0,
        "down": 1,
        "z": 0
    }]
```
> where you can see that even other collapse operator are zero, I still should define them like that.


In [7]:
# N = 6 # Turn on maximum excitation level
N = None # Turn off the maximum excitation level

w = [6.3, 5.85, 6.2] # Qubit frequency GHz
q_dim = [6 for _ in range(len(w))] # Dimension of each qubit
alpha =[-0.3, -0.3, -0.2] # Anharmonicity

r_dic = {
    "r12": 0.05,
    "r13": -0.005,
    "r23": 0.05
}

# r_freq_dependency = False # True / False, turn on / off the frequency dependency of coupling strength g12 = r12 * sqrt(w1 * w2)

r = tools.r2matrix(r_dic, w) # Coupling strength


gamma_list = [
    {
        "up": 0.005,
        "down": 0.1,
        "z": 0.001
    },
    {
        "up": 0,
        "down": 0,
        "z": 0
    },
    {
        "up": 0.005,
        "down": 0.1,
        "z": 0.001
    }
]  # Define the collapse operators: Gamma up, Gamma down, Gamma z

# Set up system class
_system = qs.qubit_system(N, q_dim, w, alpha, r, gamma_list)

# Get system Hamiltonian
_system.H

Quantum object: dims = [[216], [216]], shape = (216, 216), type = oper, isherm = True
Qobj data =
[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.

### Mathematical formulism

We use a Transmon-Coupler-Transmon system to do a demonstration. The system Hamiltonian is given by
$$
H = H_{q1} + H_{q2} + H_{c} + V_{1c} + V_{2c} + V_{12}
$$

where
$$
H_{q1} = \omega_{1} a^\dagger_1 a_1 + \frac{\alpha_1}{2} a^\dagger_1 a^\dagger_1 a_1 a_1
$$
$$
H_{q1} = \omega_{2} a^\dagger_2 a_2 + \frac{\alpha_2}{2} a^\dagger_2 a^\dagger_2 a_2 a_2
$$
$$
H_{q1} = \omega_{c} b^\dagger b + \frac{\alpha_c}{2} b^\dagger b^\dagger b b
$$
$$
V_{1c} =  g_{1c}(a^\dagger_1 + a_1)(b^\dagger + b)
$$
$$
V_{2c} =  g_{2c}(a^\dagger_2 + a_2)(b^\dagger + b)
$$
$$
V_{12} =  g_{12}(a^\dagger_1 + a_1)(a^\dagger_2 + a_2)
$$

For collapse operator, we have three types:
$$
c_+ = \sqrt{\gamma_\uparrow} a^\dagger
$$
$$
c_- = \sqrt{\gamma_\downarrow} a
$$
$$
c_z = \sqrt{\frac{\gamma_z}{2}} a^\dagger a 
$$

### Get energy, eigenstate, index of eigenstate

we use the method inside your `_system`.

`_system.get_eigenstates_energy((n,l,m))`, where `(n,l,m)` is a tuple. The returned value of this method is `state_nlm, E_nlm, index_nlm`, corresponding to the eigenstate in `Qobj` form, eigenenergy, and index of the eigenstate. 

In [12]:
state_000, E_000, index_000 = _system.get_eigenstates_energy((0,0,0))
E_000

18.123216725306463

## Single qubit

For single qubit, the defining method are the same, we could just let the interaction to be zero `r=0`

In [13]:
N =7 # Turn on maximum excitation level
# N = None # Turn off the maximum excitation level

w = [6] # Qubit frequency
q_dim = [6 for _ in range(len(w))] # Dimension of each qubit
alpha =[-0.3] # Anharmonicity
r = 0
gamma_list = None

# Set up system class
_system = qs.qubit_system(N, q_dim, w, alpha, r, gamma_list)
# Or
_system = qs.qubit_system(N, q_dim, w, alpha)

# Get system Hamiltonian
_system.H

Quantum object: dims = [[6], [6]], shape = (6, 6), type = oper, isherm = True
Qobj data =
[[  0.           0.           0.           0.           0.
    0.        ]
 [  0.          37.69911184   0.           0.           0.
    0.        ]
 [  0.           0.          73.51326809   0.           0.
    0.        ]
 [  0.           0.           0.         107.44246875   0.
    0.        ]
 [  0.           0.           0.           0.         139.48671382
    0.        ]
 [  0.           0.           0.           0.           0.
  169.64600329]]

### Get energy, eigenstate, index of eigenstate

we use the method inside your `_system`.

`_system.get_eigenstates_energy((n,))`, where `(n,)` is a tuple. The returned value of this method is `state_n, E_n, index_n`, corresponding to the eigenstate in `Qobj` form, eigenenergy, and index of the eigenstate. 

In [14]:
state_1, E_1, index_1 = _system.get_eigenstates_energy((1,))
state_1

Quantum object: dims = [[6], [1]], shape = (6, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]